In [5]:
import logging
import os
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

In [6]:

from GenreFeatureData import (
    GenreFeatureData,
)  # local python class with Audio feature extraction (librosa)

In [9]:

# set logging level
logging.getLogger("tensorflow").setLevel(logging.ERROR)

genre_features = GenreFeatureData()

In [10]:
if (
    os.path.isfile(genre_features.train_X_preprocessed_data)
    and os.path.isfile(genre_features.train_Y_preprocessed_data)
    and os.path.isfile(genre_features.dev_X_preprocessed_data)
    and os.path.isfile(genre_features.dev_Y_preprocessed_data)
    and os.path.isfile(genre_features.test_X_preprocessed_data)
    and os.path.isfile(genre_features.test_Y_preprocessed_data)
):
    print("Preprocessed files exist, deserializing npy files")
    genre_features.load_deserialize_data()
else:
    print("Preprocessing raw audio files")
    genre_features.load_preprocess_data()

print("Training X shape: " + str(genre_features.train_X.shape))
print("Training Y shape: " + str(genre_features.train_Y.shape))
print("Dev X shape: " + str(genre_features.dev_X.shape))
print("Dev Y shape: " + str(genre_features.dev_Y.shape))
print("Test X shape: " + str(genre_features.test_X.shape))
print("Test Y shape: " + str(genre_features.test_Y.shape))

Preprocessed files exist, deserializing npy files
Training X shape: (420, 128, 33)
Training Y shape: (420, 8)
Dev X shape: (120, 128, 33)
Dev Y shape: (120, 8)
Test X shape: (60, 128, 33)
Test Y shape: (60, 8)


In [11]:

input_shape = (genre_features.train_X.shape[1], genre_features.train_X.shape[2])
print("Build LSTM RNN model ...")
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=genre_features.train_Y.shape[1], activation="softmax"))

print("Compiling ...")
# Keras optimizer defaults:
# Adam   : lr=0.001, beta_1=0.9,  beta_2=0.999, epsilon=1e-8, decay=0.
# RMSprop: lr=0.001, rho=0.9,                   epsilon=1e-8, decay=0.
# SGD    : lr=0.01,  momentum=0.,                             decay=0.
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()


Build LSTM RNN model ...
Compiling ...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 128)          82944     
                                                                 
 lstm_1 (LSTM)               (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
Total params: 103,816
Trainable params: 103,816
Non-trainable params: 0
_________________________________________________________________


In [12]:
print("Training ...")
batch_size = 35  # num of training examples per minibatch
num_epochs = 400
model.fit(
    genre_features.train_X,
    genre_features.train_Y,
    batch_size=batch_size,
    epochs=num_epochs,
)

Training ...
Epoch 1/400
12/12 [==============================] - 4s 173ms/step - loss: 1.8869 - accuracy: 0.2167
Epoch 2/400
12/12 [==============================] - 3s 238ms/step - loss: 1.6687 - accuracy: 0.3333
Epoch 3/400
12/12 [==============================] - 3s 271ms/step - loss: 1.5126 - accuracy: 0.4190
Epoch 4/400
12/12 [==============================] - 3s 256ms/step - loss: 1.3491 - accuracy: 0.5095
Epoch 5/400
12/12 [==============================] - 3s 255ms/step - loss: 1.2416 - accuracy: 0.5119
Epoch 6/400
12/12 [==============================] - 3s 277ms/step - loss: 1.2308 - accuracy: 0.5167
Epoch 7/400
12/12 [==============================] - 3s 251ms/step - loss: 1.1251 - accuracy: 0.5429
Epoch 8/400
12/12 [==============================] - 3s 272ms/step - loss: 1.0676 - accuracy: 0.6071
Epoch 9/400
12/12 [==============================] - 4s 295ms/step - loss: 1.0817 - accuracy: 0.5881
Epoch 10/400
12/12 [==============================] - 3s 284ms/step - loss: 1.

In [13]:
print("\nValidating ...")
score, accuracy = model.evaluate(
    genre_features.dev_X, genre_features.dev_Y, batch_size=batch_size, verbose=1
)
print("Dev loss:  ", score)
print("Dev accuracy:  ", accuracy)


Validating ...
4/4 [==============================] - 2s 106ms/step - loss: 0.9991 - accuracy: 0.6750
Dev loss:   0.999144434928894
Dev accuracy:   0.675000011920929


In [14]:
print("\nTesting ...")
score, accuracy = model.evaluate(
    genre_features.test_X, genre_features.test_Y, batch_size=batch_size, verbose=1
)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)


Testing ...
2/2 [==============================] - 0s 158ms/step - loss: 1.5841 - accuracy: 0.6500
Test loss:   1.5840755701065063
Test accuracy:   0.6499999761581421


In [16]:
# Creates a HDF5 file 'lstm_genre_classifier.h5'
model_filename = "lstm_genre_classifier_lstm.h5"
print("\nSaving model: " + model_filename)
model.save(model_filename)


Saving model: lstm_genre_classifier_lstm.h5


In [21]:
# Creates a json file
print("creating .json file....")
model_json = model.to_json()
f = open("./lstm_genre_classifier_lstm.json","w+")
f.write(model_json)
f.close()

creating .json file....


In [2]:
"An example of predicting a music genre from a custom audio file"
import librosa
import logging
import sys
import numpy as np
from keras.models import model_from_json

In [3]:
from GenreFeatureData import (
    GenreFeatureData,
)  # local python class with Audio feature extraction and genre list

# set logging level
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [6]:


def load_model(model_path, weights_path):
    "Load the trained LSTM model from directory for genre classification"
    with open(model_path, "r") as model_file:
        trained_model = model_from_json(model_file.read())
    trained_model.load_weights(weights_path)
    trained_model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return trained_model


def extract_audio_features(file):
    "Extract audio features from an audio file for genre classification"
    timeseries_length = 128
    features = np.zeros((1, timeseries_length, 33), dtype=np.float64)

    y, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
    spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=512)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=512)

    features[0, :, 0:13] = mfcc.T[0:timeseries_length, :]
    features[0, :, 13:14] = spectral_center.T[0:timeseries_length, :]
    features[0, :, 14:26] = chroma.T[0:timeseries_length, :]
    features[0, :, 26:33] = spectral_contrast.T[0:timeseries_length, :]
    return features

def get_genre(model, music_path):
    "Predict genre of music using a trained model"
    prediction = model.predict(extract_audio_features(music_path))
    predict_genre = GenreFeatureData().genre_list[np.argmax(prediction)]
    return predict_genre


In [20]:
import os
PATH = "C:\\Users/A0826873/Bits/DL/Assignment-2/audio/metal_music.mp3"

MODEL = load_model("./lstm_genre_classifier_lstm.json", "./lstm_genre_classifier_lstm.h5")
GENRE = get_genre(MODEL, PATH)
print("Model predict: {}".format(GENRE))


1/1 [==============================] - 0s 408ms/step
Model predict: reggae
